# Camada Gold:
---

## Objetivo: 

Criar uma view analítica a partir das views `pedidos_silver`, `pagamentos_silver` e `clientes_silver`, agregando os pedidos por cliente para o cálculo das métricas de Recência, Frequência e Valor (RFV). A partir dessas métricas, os clientes são classificados em categorias, recebendo notas de A a C para cada dimensão. A combinação dessas notas permite identificar perfis de clientes, sendo AAA o cliente de maior valor para o negócio e CCC o de menor geração de receita.

### Fonte:

`workspace.olist_silver.pedidos_silver`

`workspace.olist_silver.pagamentos_silver`

`workspace.olist_silver.clientes_silver`

### Destino:

`workspace.olist_gold.clientes_rfv_base`

`workspace.olist_gold.clientes_rfv`

In [0]:
%sql
WITH data_ref AS (
    SELECT max(order_purchase_timestamp) AS ref_date
    FROM workspace.olist_silver.pedidos_silver
),

rfv_base AS (
    SELECT
        c.customer_unique_id,

        DATEDIFF(
            (SELECT ref_date FROM data_ref),
            MAX(p.order_purchase_timestamp)
        ) AS recency,

        COUNT(DISTINCT p.order_id) AS frequency,

        SUM(pg.payment_value) AS monetary_value

    FROM workspace.olist_silver.pedidos_silver p

    JOIN workspace.olist_silver.clientes_silver c
        ON p.customer_id = c.customer_id

    LEFT JOIN workspace.olist_silver.pagamentos_silver pg
        ON p.order_id = pg.order_id

    WHERE p.order_status = 'delivered'

    GROUP BY c.customer_unique_id
)

SELECT * FROM rfv_base;

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.olist_gold.clientes_rfv_base AS
WITH data_ref AS (
    SELECT max(order_purchase_timestamp) AS ref_date
    FROM workspace.olist_silver.pedidos_silver
),

rfv_base AS (
    SELECT
        c.customer_unique_id,

        DATEDIFF(
            (SELECT ref_date FROM data_ref),
            MAX(p.order_purchase_timestamp)
        ) AS recency,

        COUNT(DISTINCT p.order_id) AS frequency,

        SUM(pg.payment_value) AS monetary_value

    FROM workspace.olist_silver.pedidos_silver p

    JOIN workspace.olist_silver.clientes_silver c
        ON p.customer_id = c.customer_id

    LEFT JOIN workspace.olist_silver.pagamentos_silver pg
        ON p.order_id = pg.order_id

    WHERE p.order_status = 'delivered'

    GROUP BY c.customer_unique_id
)

SELECT * FROM rfv_base;

In [0]:
%sql
WITH scored AS (
    SELECT
        *,
        ntile(3) OVER (ORDER BY monetary_value DESC) AS v_ntile
    FROM workspace.olist_gold.clientes_rfv_base
)
SELECT
    customer_unique_id,
    recency,
    frequency,
    monetary_value,

    CASE WHEN recency <= 90 THEN 'A' WHEN recency <= 180 THEN 'B' ELSE 'C' END AS R_score,
    CASE WHEN frequency >= 5 THEN 'A' WHEN frequency >= 2 THEN 'B' ELSE 'C' END AS F_score,
    CASE v_ntile WHEN 1 THEN 'A' WHEN 2 THEN 'B' ELSE 'C' END AS V_score,

    concat(
        CASE WHEN recency <= 90 THEN 'A' WHEN recency <= 180 THEN 'B' ELSE 'C' END,
        CASE WHEN frequency >= 5 THEN 'A' WHEN frequency >= 2 THEN 'B' ELSE 'C' END,
        CASE v_ntile WHEN 1 THEN 'A' WHEN 2 THEN 'B' ELSE 'C' END
    ) AS RFV_class

FROM scored;

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.olist_gold.clientes_rfv AS
WITH scored AS (
    SELECT
        *,
        ntile(3) OVER (ORDER BY monetary_value DESC) AS v_ntile
    FROM workspace.olist_gold.clientes_rfv_base
)
SELECT
    customer_unique_id,
    recency,
    frequency,
    monetary_value,

    CASE WHEN recency <= 90 THEN 'A' WHEN recency <= 180 THEN 'B' ELSE 'C' END AS R_score,
    CASE WHEN frequency >= 5 THEN 'A' WHEN frequency >= 2 THEN 'B' ELSE 'C' END AS F_score,
    CASE v_ntile WHEN 1 THEN 'A' WHEN 2 THEN 'B' ELSE 'C' END AS V_score,

    concat(
        CASE WHEN recency <= 90 THEN 'A' WHEN recency <= 180 THEN 'B' ELSE 'C' END,
        CASE WHEN frequency >= 5 THEN 'A' WHEN frequency >= 2 THEN 'B' ELSE 'C' END,
        CASE v_ntile WHEN 1 THEN 'A' WHEN 2 THEN 'B' ELSE 'C' END
    ) AS RFV_class

FROM scored;